In [1]:
import numpy
import pandas
import requests

from time import sleep
from tqdm.notebook import tqdm, trange

## ML Class II - Otimização

No problema proposto, devemos determinar quais os ângulos ótimos de configuração de uma antena para que o _ganho_ produzido por tal arrranjo seja o maior possível. Mais precisamente, o ganho é dado por uma função $f: \mathbb{R}^6 \rightarrow \mathbb{R}$ e queremos encontrar o ponto $\textbf{X} = (\phi_1, \theta_1, \phi_2, \theta_2, \phi_3, \theta_3)$ no seu domínio tal que o valor de $f$ é máximo.

Começamos definindo a função:

In [2]:
def f(x):
    """
    Recebe um numpy.array([phi1, theta1, phi2, theta2, phi3, theta3]) indicando os ângulos e retorna o ganho.
    """
    values = tuple(x)
    request_url = f'http://localhost:8080/antenna/simulate?phi1=%.0f&theta1=%.0f&phi2=%.0f&theta2=%.0f&phi3=%.0f&theta3=%.0f' % values
    response = requests.get(request_url).content.decode('utf-8')
    return float(response.split('\n')[0])

# Abordagem I - Subida da Encosta

In [3]:
EPS = 0.1
NUM = 1000

In [4]:
def hill_climbing(f, x_init):

    x = x_init
    dx = f(x + numpy.full_like(x, 1)) - f(x)
    
    while (x != x_init).all():
        x_init = x
        x += int(dx)
        dx = f(x + numpy.full_like(x, 1)) - f(x)
        
    return x, f(x)

In [5]:
results = []

for _ in trange(NUM):
    x = numpy.random.randint(low=0, high=360, size=6)
    _, local_max = hill_climbing(f=f, x_init=x)
    results.append(numpy.append(x, local_max))
    
results = numpy.array(results)

In [6]:
results = pandas.DataFrame(results, columns=['phi1', 'theta1', 'phi2', 'theta2', 'phi3', 'theta3', 'f_max']).sort_values(by='f_max', ascending=False).reset_index(drop=True)
results

,phi1,theta1,phi2,theta2,phi3,theta3,f_max
0,208.0,149.0,348.0,41.0,165.0,199.0,24.330050
1,220.0,142.0,193.0,335.0,162.0,148.0,17.037643
2,255.0,213.0,36.0,231.0,205.0,154.0,16.983242
3,207.0,127.0,304.0,90.0,153.0,201.0,15.604010
4,213.0,110.0,54.0,227.0,166.0,161.0,14.850498
...,...,...,...,...,...,...,...
995,347.0,271.0,41.0,32.0,324.0,136.0,-11.925629
996,138.0,71.0,43.0,38.0,168.0,53.0,-12.803913
997,145.0,275.0,3.0,93.0,137.0,261.0,-13.113412
998,145.0,259.0,5.0,65.0,108.0,67.0,-13.412605


In [7]:
# 1) Possibilidade de escolher um intervalo melhor para chutar os pontos

# Abordagem II - Algoritmo Genético

In [8]:
class Individual:
    
    mutation_rate = 0.01
    
    def __init__(self, phi1=None, theta1=None, phi2=None, theta2=None, phi3=None, theta3=None):
        
        self.phi1 = numpy.random.randint(low=0, high=360) if phi1 is None else phi1
        self.theta1 = numpy.random.randint(low=0, high=360) if theta1 is None else theta1
        
        self.phi2 = numpy.random.randint(low=0, high=360) if phi2 is None else phi2
        self.theta2 = numpy.random.randint(low=0, high=360) if theta2 is None else theta2
        
        self.phi3 = numpy.random.randint(low=0, high=360) if phi3 is None else phi3
        self.theta3 = numpy.random.randint(low=0, high=360) if theta3 is None else theta3
        
    
    def get_dna(self):
        return [self.phi1, self.theta1, self.phi2, self.theta2, self.phi3, self.theta3]
    
    def mutate(self):
        
        mutation = numpy.random.binomial(1, self.mutation_rate, size=6)
        
        self.phi1 = numpy.random.randint(low=0, high=360) if mutation[0] else self.phi1
        self.theta1 = numpy.random.randint(low=0, high=360) if mutation[1] else self.theta1
        
        self.phi2 = numpy.random.randint(low=0, high=360) if mutation[2] else self.phi2
        self.theta2 = numpy.random.randint(low=0, high=360) if mutation[3] else self.theta2
        
        self.phi3 = numpy.random.randint(low=0, high=360) if mutation[4] else self.phi3
        self.theta3 = numpy.random.randint(low=0, high=360) if mutation[5] else self.theta3
        
    def __str__(self):
        return '%d|%d|%d|%d|%d|%d' % tuple(self.get_dna())
        
        
class Population:
    
    best_fitness = -numpy.Inf
    best_arg = numpy.empty(shape=6)
    
    def __init__(self, size):
        
        self.size = size
        self.individuals = [Individual() for _ in range(size)]
        
    def fit_select(self, fitness_func, fit_size):
        
        fit_size = int(fit_size * self.size) if type(fit_size) == float else fit_size
        fit_size = fit_size + 1 if fit_size % 2 else fit_size
                
        pop_fitness = []
        
        for ind in self.individuals:
            
            curr_fitness = fitness_func(numpy.array(ind.get_dna()))
            
            if curr_fitness - self.best_fitness > 0:
                self.best_fitness = curr_fitness
                self.best_arg = ind.get_dna()
            
            pop_fitness.append(curr_fitness)
            
            sleep(0.001)
        
        pop_fitness = numpy.array(pop_fitness)
        
        self.best_fitness = pop_fitness.max()
        
        pop_fitness = (pop_fitness - pop_fitness.min()) / (pop_fitness.max() - pop_fitness.min())
        pop_fitness = pop_fitness / pop_fitness.sum()
        
        selected_individuals = numpy.random.choice([i for i in range(self.size)], size=fit_size, replace=False, p=pop_fitness)
        
        self.individuals = [self.individuals[i] for i in selected_individuals]
            
    def reproduce(self):
        
        new_individuals = []
        
        for _ in range(self.size - len(self.individuals)):
            
            par_idxs = numpy.random.randint(low=0, high=len(self.individuals)-1, size=2)
            
            dna = self.crossover(par_idxs[0], par_idxs[1])
            
            new_ind = Individual(phi1=dna[0], theta1=dna[1], phi2=dna[2], theta2=dna[3], phi3=dna[4], theta3=dna[5])
            
            new_ind.mutate()
            
            new_individuals.append(new_ind)
            
        self.individuals = self.individuals + new_individuals
        
        
    def crossover(self, par1_idx, par2_idx):
        
        par1_dna = self.individuals[par1_idx].get_dna()
        par2_dna = self.individuals[par2_idx].get_dna()
        
        split_point = numpy.random.randint(low=1, high=4)
        
        return numpy.concatenate((par1_dna[:split_point], par2_dna[split_point:]))
    
    def __str__(self):
        return '\n'.join([str(ind) for ind in self.individuals])

In [9]:
INIT_POP = 500
FIT_SIZE = 0.25
TOTAL_GEN = 20

In [ ]:
pop = Population(size=INIT_POP)

t = trange(TOTAL_GEN, desc='Generation: Best Fitness: ', leave=True)

for i in t:
    pop.fit_select(fitness_func=f, fit_size=FIT_SIZE)
    pop.reproduce()
    t.set_description('Generation: %d Best Fitness: %.2f' % (i, pop.best_fitness))
    t.refresh()

In [ ]:
result_df = pandas.DataFrame(numpy.column_stack((numpy.array([ind.get_dna() for ind in pop.individuals]), numpy.array([f(numpy.array(ind.get_dna())) for ind in pop.individuals]))) , columns=['phi1', 'theta1', 'phi2', 'theta2', 'phi3', 'theta3', 'f_max']).sort_values(by='f_max', ascending=False).reset_index(drop=True)
result_df